In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from datetime import datetime, timedelta
import time


import warnings
warnings.filterwarnings('ignore')

In [2]:
#Allows us to run on different computers without a problem

path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': r"C:\Users\yarin\chromedriver.exe", 'yair':'C:\Windows\chromdriver.exe' }


In [3]:
def scroll(driver):
    time.sleep(3)
    time_to_wait = 25 
    start = time.time()
    while True:
        driver.execute_script('scrollBy(0,500)')
        time.sleep(0.5)
        if start + time_to_wait < time.time():
            break
        
    try:
        for i in range(0,2):
            button=driver.find_element(by =By.CSS_SELECTOR,value =f'[data-stid="show-more-results"]')
            button.click()
            time.sleep(3)
    except:
        pass


In [4]:
Unsuccessful_list  = [] # A list that saves the names of the hotels that have a problem in order to optimize and not try to extract information again

def single_search(url,driver, Snapshot_date, checking_date_type, checkout_date_type):
    
    hotels = []
    hotels_name = []
    index = 1
    driver.get(url)  
    scroll(driver)
    text_boxes_list = driver.find_elements(by = By.XPATH , value='.//*[@class="uitk-spacing uitk-spacing-margin-blockstart-three"]')
    for t_box in text_boxes_list:
        if len(hotels)>= 100:
            break
        hotel = {'Snapshot' : None ,'Index':None,'Hotel Name':None,'TTT':None,'LOS':None ,"Grade":None,
         "Num of Reviews":None,"Curr Price":None,"Original Price":None, 'Left at this Price': None,
         'Included': None, 'Refundable': None,
         'Pay Later': None, 'Option Member': None}
        try:

            hotel['Snapshot'] = Snapshot_date
            delta_TTT =  checking_date_type - Snapshot_date
            hotel['TTT'] = delta_TTT.days
            hotel['LOS'] = (checkout_date_type - checking_date_type).days
            hotel['Index'] = index
            
            name = t_box.find_element(by= By.CLASS_NAME , value="uitk-heading").text
            hotel['Hotel Name'] = name
            if hotel['Hotel Name'] in hotels_name:
                continue
            if (name in Unsuccessful_list):#Avoid hotels that we cannot extract
                    continue
            score_count = t_box.find_element(By.XPATH, './/*[@class="uitk-layout-flex uitk-layout-flex-flex-direction-column uitk-layout-flex-gap-three"]').text
            hotel['Num of Reviews'] = score_count.split('\n')[::-1][1]
            if ("Fully refundable" in score_count):
                hotel['Refundable'] = True
            else:
                hotel['Refundable'] = False
            
            if ("Reserve now, pay later" in score_count):
                hotel['Pay Later'] = True
            else:
                hotel['Pay Later'] = False

            score = t_box.find_element(By.CLASS_NAME, 'uitk-spacing-padding-inlineend-half').text
            hotel['Grade'] = score 
            
            price = t_box.find_element(By.CSS_SELECTOR, "div[data-test-id='price-summary-message-line']").text.split('$')
            hotel['Curr Price'] = price[-1]
            if len(price) > 3:
                hotel['Original Price'] = price[1].split('\n')[0]

            try:
                hotel['Left at this Price'] = t_box.find_element(By.CLASS_NAME, 'uitk-badge-text').text
            except:
                pass
            try:
                included_sel_type = t_box.find_elements(By.XPATH, './/*[@class="uitk-text truncate uitk-type-200 uitk-text-default-theme"]')
                included_list_type = []
                for i in included_sel_type:
                    included_list_type.append(i.text)
                hotel['Included'] = included_list_type
            except:
                pass
           
            try:
                x = t_box.find_element(By.XPATH, './/*[@class="uitk-icon uitk-icon-leading uitk-icon-medium"]').text
                hotel['Option Member'] = True 
            except:
                hotel['Option Member'] = False
            
            index = index + 1
            hotels_name.append(hotel['Hotel Name'])
            hotels.append(hotel)
        except:
            print('error' , name)
            Unsuccessful_list.append(name)
    
    return pd.DataFrame.from_dict(hotels)

In [5]:
# Get today's date
now = datetime.now()
Snapshot_date =now
# Get end date 35 days from now
end_date = now + timedelta(days=35)

# Create a list of all dates between today and end date
date_list = []
while now <= end_date:
    date_list.append(now.strftime('%Y-%m-%d'))
    now += timedelta(days=1)


#Creates a list whose every member is a list of checking and checkout time

combinations_list =[]
for i in range(1,len(date_list)-5):
    for j in range(1,6):
        combinations = [date_list[i],date_list[i+j]]
        combinations_list.append(combinations)

print(combinations_list , len(combinations_list)) 

[['2023-03-04', '2023-03-05'], ['2023-03-04', '2023-03-06'], ['2023-03-04', '2023-03-07'], ['2023-03-04', '2023-03-08'], ['2023-03-04', '2023-03-09'], ['2023-03-05', '2023-03-06'], ['2023-03-05', '2023-03-07'], ['2023-03-05', '2023-03-08'], ['2023-03-05', '2023-03-09'], ['2023-03-05', '2023-03-10'], ['2023-03-06', '2023-03-07'], ['2023-03-06', '2023-03-08'], ['2023-03-06', '2023-03-09'], ['2023-03-06', '2023-03-10'], ['2023-03-06', '2023-03-11'], ['2023-03-07', '2023-03-08'], ['2023-03-07', '2023-03-09'], ['2023-03-07', '2023-03-10'], ['2023-03-07', '2023-03-11'], ['2023-03-07', '2023-03-12'], ['2023-03-08', '2023-03-09'], ['2023-03-08', '2023-03-10'], ['2023-03-08', '2023-03-11'], ['2023-03-08', '2023-03-12'], ['2023-03-08', '2023-03-13'], ['2023-03-09', '2023-03-10'], ['2023-03-09', '2023-03-11'], ['2023-03-09', '2023-03-12'], ['2023-03-09', '2023-03-13'], ['2023-03-09', '2023-03-14'], ['2023-03-10', '2023-03-11'], ['2023-03-10', '2023-03-12'], ['2023-03-10', '2023-03-13'], ['2023-03

In [10]:
#fake for bot
driver = webdriver.Chrome(path['roy'])
url = "https://www.expedia.com/Hotel-Search?adults=2&children=0&d1=%5B%272023%27%2C%20%2703%27%2C%20%2703%27%5D&d2=%5B%272023%27%2C%20%2703%27%2C%20%2704%27%5D&destination=New%20York%20%28and%20vicinity%29%2C%20New%20York%2C%20United%20States%20of%20America&endDate=2023-03-17&regionId=178293&semdtl=&sort=RECOMMENDED&startDate=2023-03-16&theme=&useRewards=false&userIntent="
driver.get(url) 
driver.maximize_window()
driver.get(url) 

In [11]:
# for dev
combinations_list = combinations_list[0:2:] 
# END - for dev


city = "New%20York%20%28and%20vicinity%29%2C%20New%20York%2C%20United%20States%20of%20America"
city_latlong = 'latLong=40.75668%2C-73.98647'
city_region_id = '178293'
numOfAdults=2
numOfChild=0
numOfRooms=1
expedia_df = pd.DataFrame(columns=['Snapshot' ,'Index','Hotel Name','TTT','LOS',"Grade",
                                "Num of Reviews","Curr Price","Original Price", 'Left at this Price',
                                'Included', 'Refundable','Pay Later', 'Option Member'])
for comb in combinations_list:
    checking_date_type = datetime.strptime(comb[0], '%Y-%m-%d')
    checkout_date_type = datetime.strptime(comb[1], '%Y-%m-%d')
    url = f"https://www.expedia.com/Hotel-Search?adults={numOfAdults}&d1={comb[0]}&d2={comb[1]}&destination={city}&endDate={comb[1]}&latLong={city_latlong}&regionId={city_region_id}&rooms={numOfRooms}&semdtl=&sort=RECOMMENDED&startDate={comb[0]}&theme=&useRewards=false&userIntent="
    single_search_df  = single_search(url,driver , Snapshot_date, checking_date_type, checkout_date_type)
    expedia_df = pd.concat([expedia_df, single_search_df])

    
#driver.quit()

error Discover Boston
error Sky Hotel Flushing/LaGuardia Airport
error Discover Washington, DC
error Find your luxury escape
error Latisha house


In [12]:
len(expedia_df)

200

In [13]:
expedia_df.head(60)

,Snapshot,Index,Hotel Name,TTT,LOS,Grade,Num of Reviews,Curr Price,Original Price,Left at this Price,Included,Refundable,Pay Later,Option Member
0,2023-03-03 17:13:44.747469,1,"PUBLIC, an Ian Schrager hotel",0,1,8.0,"(1,450 reviews)",349,None,VIP Access,[],False,False,True
1,2023-03-03 17:13:44.747469,2,The Westin New York Grand Central,0,1,8.6,"(1,170 reviews)",204,None,We have 5 left at,[],False,False,False
2,2023-03-03 17:13:44.747469,3,Sofitel New York,0,1,8.8,"(1,002 reviews)",242,269,10% off,[],False,False,False
3,2023-03-03 17:13:44.747469,4,Hilton Garden Inn NYC Financial Center/Manhatt...,0,1,8.6,"(1,002 reviews)",162,190,None,[],True,True,True
4,2023-03-03 17:13:44.747469,5,"Club Quarters Hotel, World Trade Center",0,1,8.6,"(1,093 reviews)",179,None,None,[],True,True,True
5,2023-03-03 17:13:44.747469,6,"The Jewel, a Club Quarters Hotel, Opposite Roc...",0,1,9.2,"(1,222 reviews)",209,None,None,[],True,True,True
6,2023-03-03 17:13:44.747469,7,Park Central Hotel New York,0,1,8.0,"(1,890 reviews)",224,249,None,[],False,False,True
7,2023-03-03 17:13:44.747469,8,Four Points By Sheraton New York Downtown,0,1,7.4,"(1,002 reviews)",169,None,We have 5 left at,[],False,False,False
8,2023-03-03 17:13:44.747469,9,The Shelburne Sonesta New York,0,1,8.2,"(1,004 reviews)",157,None,VIP Access,[],False,False,False
9,2023-03-03 17:13:44.747469,10,Freehand New York,0,1,8.2,"(1,003 reviews)",179,None,We have 1 left at,[],True,False,False


In [14]:
expedia_df.groupby(['TTT','LOS']).count()

Snapshot  Index  Hotel Name  Grade  Num of Reviews  Curr Price  \
TTT LOS                                                                   
0   1         100    100         100    100             100         100   
    2         100    100         100    100             100         100   

         Original Price  Left at this Price  Included  Refundable  Pay Later  \
TTT LOS                                                                        
0   1                23                  49       100         100        100   
    2                25                  50       100         100        100   

         Option Member  
TTT LOS                 
0   1              100  
    2              100

In [ ]:
file_name = "expedia_hotels.csv"
if os.path.exists(file_name):#if the "Booking_hotels.csv" exists
    print("exists")
    expedia_df.to_csv(file_name,mode = 'a' , index= False ,header=False)#append without the header
else:
    print("new")
    expedia_df.to_csv(file_name,mode = 'a' , index= False ,header=True)#append with the header